In [10]:
import numpy as np
import matplotlib.pyplot as plt

import sys, os

sys.path.insert(0, '/Users/sam/PycharmProjects/Quantum-Control/NewModule')
from arc_sam import *  # Import ARC (Alkali Rydberg Calculator)
atom = Calcium40()
calc = StarkMap(atom)

In [11]:
n = 35
l = 3
j = 3
mj = 0
s = 0
nmin = n - 1
nmax = n + 1
lmax = 5
calc.defineBasis(n,l,j,mj,nmin,nmax,lmax,s=s)
efield = [0,500,1000,2000]
calc.diagonalise(efield, upTo=-1)

In [12]:
basis_size = len(calc.basisStates)
new_basis = np.zeros_like(calc.basisStates)
for k in range(basis_size):
    new_basis[k] = calc.basisStates[calc.index_new_basis[k]]


In [14]:
# soit un atome de ca dans un etat 35F^3, on se demande ce qu'il se passe si on applique plusieurs pulse. Ici on ignore la partie temporelle du système.
# comment on va fonctionner ?
# 1. on applique le premier pulse F1 : ce pulse va impliquer un effet Stark, donc melange des niveaux atomiques (niveau stark)
# 2. eteindre le champ electrique F1 : après avoir éteind directement le champ, les etats Stark vont se projeter directement sur les etats atomiques
# 3. application d'un nouveau champ electrique F2 : cette fois, on prend les coefficients des niveaux Stark comme etat initial du système, et on regarde l'evolution des etats
# eteindre le champ electrique F2.
# etc....

#_____________________________1.
#on va faire un peu de recyclage et réutiliser les valeurs calculer précédemment. efield = 0.5,1.0,2.0
initial_psi = np.zeros(len(calc.basisStates))
# on doit aller pacer un 1 dans l'état qui correspond à notre état 35F3. on a déjà diagonaliser donc on doit aller chercher dans index_new basis.
initial_psi[calc.index_new_basis[calc.indexOfCoupledState]] = 1
total_population = np.zeros(4)
for k in range(len(calc.basisStates)):
    total_population[0] += initial_psi[k]
#2.____________________________

compo_stark_first_pulse = calc.composition[1] #composition give the decomposition of each stark state in atomic state
#composition has all the stark level computed. Knowing we are in the specific stark state corresponding to the intial state, we can go take it in composition
stark_state = compo_stark_first_pulse[calc.index_new_basis[calc.indexOfCoupledState]]
projected_atomic_population = np.zeros(len(calc.basisStates))
for k in range(len(calc.basisStates)):
    to_index = stark_state[k][1]
    c_nl = stark_state[k][0]
    projected_atomic_population[to_index] += c_nl
stark_state_first_pulse = projected_atomic_population
abs_projected_atomic_population = np.abs(projected_atomic_population)
for k in range(basis_size):
    total_population[1] += abs_projected_atomic_population[k] ** 2
k = 0
non_zero_index = []



In [15]:
#3.____________________________

compo_stark_second_pulse = calc.composition[2]
initial_psi = projected_atomic_population
stark_sate_second_pulse = np.zeros(len(calc.basisStates))
projected_atomic_population = np.zeros(len(calc.basisStates))
for k in range(len(calc.basisStates)):
    # the actual state is given by the formule : sum_i c_i * |i>
    # where c_i is the coefficient of the stark state in the decomposition of the initial state
    # and |i> is the ith basis state
    # for this, we have to put another c_j in the formula, because of the projection on the stark state
    if initial_psi[k] != 0:
        corresponding_stark_state = compo_stark_second_pulse[calc.index_new_basis[k]]
        for j in range(len(corresponding_stark_state)):
            stark_sate_second_pulse[corresponding_stark_state[j][1]] += corresponding_stark_state[j][0] * initial_psi[k]
    else:
        pass

projected_atomic_population = stark_sate_second_pulse
abs_projected_atomic_population = np.abs(projected_atomic_population)
for k in range(basis_size):
    total_population[2] += abs_projected_atomic_population[k] ** 2

#4______________
#third pulse

compo_stark_third_pulse = calc.composition[3]
initial_psi_third_pulse = projected_atomic_population
projected_atomic_population = np.zeros(len(calc.basisStates))
stark_state_third_pulse = np.zeros(len(calc.basisStates))
for k in range(len(calc.basisStates)):
    # the actual state is given by the formule : sum_i c_i * |i>
    # where c_i is the coefficient of the stark state in the decomposition of the initial state
    # and |i> is the ith basis state
    # for this, we have to put another c_j in the formula, because of the projection on the stark state
    if initial_psi_third_pulse[k] != 0:
        corresponding_stark_state = compo_stark_third_pulse[calc.index_new_basis[k]]
        for j in range(len(corresponding_stark_state)):
            stark_state_third_pulse[corresponding_stark_state[j][1]] += corresponding_stark_state[j][0] * initial_psi_third_pulse[k]
           
    else:
        pass
    

projected_atomic_population = stark_state_third_pulse
abs_projected_atomic_population = np.abs(projected_atomic_population)
for k in range(basis_size):
    total_population[3] += abs_projected_atomic_population[k] ** 2

In [16]:
## vérficiation NECESSAIRE
# regardons que, en effet, si on par de l'etat initial et qu'on va direct au dernier pulse, alors le résultat est différent (état final dépend bien des pulses précédents
ini_test = np.zeros(len(calc.basisStates))
ini_test[calc.index_new_basis[calc.indexOfCoupledState]] = 1
compo_test = calc.composition[3]
proj_test = np.zeros(len(calc.basisStates))
for k in range(len(calc.basisStates)):
    if ini_test[k] != 0:
        corresponding_stark_state = compo_test[calc.index_new_basis[k]]
        for j in range(len(corresponding_stark_state)):
            proj_test[corresponding_stark_state[j][1]] += corresponding_stark_state[j][0] * ini_test[k]
    else:
        pass
abs_proj_test = np.abs(proj_test)
pop_test = 0
for k in range(len(calc.basisStates)):
    pop_test += abs_proj_test[k] ** 2
print(pop_test)
print(proj_test)

1.0
[ 4.63625003e-06  4.81038342e-05 -1.52210523e-02  7.27165857e-03
 -4.94076227e-02  1.24801579e-02 -4.69995534e-05 -9.30467521e-04
 -9.90858237e-03 -6.50210031e-01  6.47272727e-01 -3.94061489e-01]


In [17]:
# Define the initial state in the atomic basis (e.g., 35F³ state)
initial_psi = np.zeros(len(calc.basisStates))
initial_psi[calc.indexOfCoupledState] = 1  # Set the initial state to the coupled state

# Get the eigenvector matrix for the first electric field (F=500 V/cm)
# Assuming calc.egvector[1] is a 1D array, reshape it into a column vector
U_F1 = np.array(calc.egvector[1]).reshape(-1, 1)  # Shape: (69, 1)

# Transform the initial state to the Stark basis
psi_stark = U_F1.T @ initial_psi  # Shape: (1, 69) @ (69,) -> (1,)

# After turning off the field, project back to the atomic basis
projected_atomic_population = U_F1 @ psi_stark  # Shape: (69, 1) @ (1,) -> (69,)

# Get the eigenvector matrix for the second electric field (F=1000 V/cm)
U_F2 = np.array(calc.egvector[2]).reshape(-1, 1)  # Shape: (69, 1)

# Use the projected atomic population as the new initial state
initial_psi_second_pulse = projected_atomic_population

# Transform to the Stark basis for the second pulse
psi_stark_F2 = U_F2.T @ initial_psi_second_pulse  # Shape: (1, 69) @ (69,) -> (1,)

# After turning off the field, project back to the atomic basis
projected_atomic_population_F2 = U_F2 @ psi_stark_F2  # Shape: (69, 1) @ (1,) -> (69,)

# Debugging output
print("Initial state (atomic basis):", initial_psi)
print("Stark state after first pulse:", psi_stark)
print("Projected atomic population after first pulse:", projected_atomic_population)
print("Stark state after second pulse:", psi_stark_F2)
print("Projected atomic population after second pulse:", projected_atomic_population_F2)

U_F3 = np.array(calc.egvector[3]).reshape(-1, 1)
initial_psi_third_pulse = projected_atomic_population
psi_stark_F3 = U_F3.T @ initial_psi_third_pulse
projected_atomic_population_F3 = U_F3 @ psi_stark_F3

print("Stark state after third pulse:", psi_stark_F3)
print("Projected atomic population after third pulse:", projected_atomic_population_F3)


Initial state (atomic basis): [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Stark state after first pulse: [0.92708408]
Projected atomic population after first pulse: [-4.67281993e-07 -1.54785593e-05 -6.25134092e-05 -5.07149566e-03
 -4.62548678e-04  3.57229819e-03  2.27147815e-03  6.13024570e-03
  7.71201494e-03  8.59484895e-01 -2.94005185e-01  1.84907131e-01]
Stark state after second pulse: [-0.89225366]
Projected atomic population after second pulse: [ 3.53794373e-06  5.79509630e-05  4.83302298e-04  9.66207207e-03
  1.72609969e-03 -7.60178336e-03  7.51261844e-03 -8.63178841e-03
 -1.56598260e-02  7.07192730e-01 -4.74721702e-01  2.64771235e-01]
Stark state after third pulse: [-0.82205901]
Projected atomic population after third pulse: [ 2.44312621e-05  1.88788718e-04  3.43689271e-03  1.70161238e-02
  5.77486598e-03 -1.65548183e-02 -1.49138664e-02  8.41620938e-03
  3.22062120e-02  5.34511016e-01 -5.38255585e-01  3.13720600e-01]


In [18]:
# Define the initial state in the atomic basis (e.g., 35F³ state)
initial_psi = np.zeros(len(calc.basisStates))
initial_psi[calc.indexOfCoupledState] = 1  # Set the initial state to the coupled state
hbar  = 1
# Get the eigenvector matrix for the first electric field (F=500 V/cm)
U_F1 = np.array(calc.egvector[1])  # Eigenvectors for F=500 V/cm (shape: (69, 69))

# Transform the initial state to the Stark basis for the first pulse
psi_stark = U_F1.T @ initial_psi  # Shape: (69, 69) @ (69,) -> (69,)

# Perform calculations in the Stark basis (e.g., time evolution, pulses, etc.)
# For example, evolve the state in time (assuming Hamiltonian is diagonal in Stark basis)
energies = np.array(calc.y[1])  # Eigenvalues (energies) for F=500 V/cm
time = 1.0  # Example time
psi_stark_evolved = psi_stark * np.exp(-1j * energies * time / hbar)

# Decompose the evolved Stark state into the atomic basis using calc.composition
projected_atomic_population = np.zeros(len(calc.basisStates))
for k, coefficient in enumerate(psi_stark_evolved):
    for comp in calc.composition[1][k]:  # Decomposition for the i-th Stark state
        atomic_state_index = comp[1]  # Index of the atomic basis state
        atomic_state_coefficient = comp[0]  # Coefficient of the atomic basis state
        projected_atomic_population[atomic_state_index] += coefficient * atomic_state_coefficient

# Debugging output
print("Initial state (atomic basis):", initial_psi)
print("Stark state after first pulse:", psi_stark)
print("Projected atomic population after evolution:", projected_atomic_population)

Initial state (atomic basis): [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
Stark state after first pulse: 0.9270840820708063
Projected atomic population after evolution: [ 0.36911452 -0.51065461 -0.78992897 -0.64958873 -0.01798436 -1.13817249
  0.35678732 -0.62025596  0.60084136  0.12670449 -0.94035563  0.19827799]


/var/folders/8l/xxl61m252sq_j4xxgx_ll1qr0000gn/T/ipykernel_69979/3803807815.py:23: ComplexWarning: Casting complex values to real discards the imaginary part
  projected_atomic_population[atomic_state_index] += coefficient * atomic_state_coefficient


In [19]:
import numpy as np

def simulate_pulses(calc, num_pulses=3):
    # Initialize state in atomic basis
    basis_size = len(calc.basisStates)
    initial_psi = np.zeros(basis_size, dtype=np.complex128)

    # Set initial state using index mapping
    initial_psi[calc.index_new_basis[calc.indexOfCoupledState]] = 1.0 + 0j

    # Store populations for each pulse
    populations = []
    total_population = []

    current_state = initial_psi.copy()

    for pulse_idx in range(num_pulses):
        # Get pulse parameters (0: 0V/cm, 1:500V/cm, 2:1000V/cm, 3:2000V/cm)
        egvector = np.array(calc.egvector[pulse_idx + 1], dtype=np.complex128)
        composition = calc.composition[pulse_idx + 1]

        # Transform to Stark basis
        psi_stark = egvector.conj().T @ current_state

        # Project back to atomic basis using composition data
        projected = np.zeros(basis_size, dtype=np.complex128)

        for stark_idx, (coeff, atomic_idx) in enumerate(composition):
            if atomic_idx < basis_size:  # Ensure index is within bounds
                projected[atomic_idx] += coeff * psi_stark[stark_idx]

        # Calculate probabilities
        prob = np.abs(projected) ** 2
        total_prob = np.sum(prob)

        # Store results
        populations.append(prob)
        total_population.append(total_prob)

        # Set new initial state for next pulse
        current_state = projected.copy()

        # Debug output
        print(f"\nAfter Pulse {pulse_idx+1} ({calc.eFieldList[pulse_idx+1]} V/cm):")
        print(f"Total population: {total_prob:.6f}")
        print(f"Max probability: {np.max(prob):.4f} at state {np.argmax(prob)}")

    return populations, total_population

# Example usage
populations, total_population = simulate_pulses(calc)

# Additional analysis
print("\nFinal populations:")
for k, (pop, total) in enumerate(zip(populations, total_population)):
    print(f"Pulse {k + 1}: Total = {total:.6f}, Max = {np.max(pop):.4f}")

# Verify conservation of probability
print("\nProbability conservation:")
print("Initial:", 1.0)
print("After pulses:", [f"{t:.6f}" for t in total_population])

ValueError: too many values to unpack (expected 2)